In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/1-3m-linkedin-jobs-and-skills-2024/job_summary.csv
/kaggle/input/1-3m-linkedin-jobs-and-skills-2024/job_skills.csv
/kaggle/input/1-3m-linkedin-jobs-and-skills-2024/linkedin_job_postings.csv


In [ ]:
summary = pd.read_csv('/kaggle/input/1-3m-linkedin-jobs-and-skills-2024/job_summary.csv')
skills = pd.read_csv('/kaggle/input/1-3m-linkedin-jobs-and-skills-2024/job_skills.csv')
postings = pd.read_csv('/kaggle/input/1-3m-linkedin-jobs-and-skills-2024/linkedin_job_postings.csv')

In [ ]:
merged_df = pd.merge(summary,skills,on='job_link',how='outer')

In [ ]:
merged_df.info()

In [ ]:
merged_df.dropna(inplace=True)

In [ ]:
df = pd.merge(merged_df, postings, on='job_link', how='outer')
df.info()

In [ ]:
df.head(5)

In [ ]:
df.duplicated().sum()

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(subset=['company','job_location'],inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df_short = df[df['job_summary'].isnull() & df['job_skills'].isnull()]
df_short.isnull().sum()

In [ ]:
df_short.head()

In [ ]:
df_new = df.drop(['job_summary','job_skills','job_link','search_city','last_processed_time','got_summary','got_ner','is_being_worked'],axis=1)
df_new.head()

In [ ]:
df_new.info()

In [ ]:
df_try = df_new.rename(columns={'job_title': 'title', 
                            'job_location': 'location', 
                            'search_country': 'country'})

In [ ]:
df_new = df_new.drop(['search_country'],axis=1)

In [ ]:
df_new.head()

In [ ]:
df_new.info()

In [ ]:
df_one = pd.DataFrame()
df_one['tags'] = df_new['job_title'] + ' ' + \
                              df_new['company'] + ' ' + \
                              df_new['job_location'] + ' ' + \
                              df_new['search_position'] + ' ' + \
                              df_new['job_level'] + ' ' + \
                              df_new['job_type']

In [ ]:
df_one.head()

In [ ]:
def remove_duplicate_words(tag_list):
    cleaned_tags = []
    for tags in tag_list:
        words = tags.split()
        unique_words = list(dict.fromkeys(words))  
        cleaned_tags.append(' '.join(unique_words))
    return cleaned_tags

# Apply the function only to the 'tags' column
df_one['new_tags'] = remove_duplicate_words(df_one['tags'])

# Drop the original 'tags' column
df_two = df_one.drop(['tags'], axis=1)

In [ ]:
df_two['new_tags'][0]

In [ ]:
df_one['tags'][0]

In [ ]:
import re
df_two['new_tags'] = df_two['new_tags'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfid = TfidfVectorizer(stop_words='english')

tfid_matrix = tfid.fit_transform(df_two['new_tags'])
print(tfid_matrix.shape)

In [ ]:
user_input = "Database Manager in India"

input_tfid = tfid.transform([user_input])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(input_tfid, tfid_matrix)

In [ ]:
print(cosine_sim)

In [ ]:
sim_scores = cosine_sim[0]
job_indices = sim_scores.argsort()[::-1]
top_n = 15
top_job_indices = job_indices[:top_n]

In [ ]:
recommended_jobs = df_two.iloc[top_job_indices]
recommended_jobs.head()

In [ ]:
job_ids = recommended_jobs.index
job_ids_list = job_ids.tolist()
job_ids_list

In [ ]:
for i in job_ids_list:
    print(df['job_link'][i])

# Model 2

In [ ]:
df.head()

In [ ]:
df_2 = df.copy()

In [ ]:
df_2['job_summary'].fillna('None',inplace=True)
df_2['job_skills'].fillna('None',inplace=True)
df_2.head()

In [ ]:
df_skill = pd.DataFrame
df_skill = df_2[['job_skills','job_title']]
df_skill.head()

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from collections import Counter


vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(df_skill['job_title'])

num_clusters = 15
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
df_skill['cluster'] = kmeans.fit_predict(X)

def get_top_skills(cluster_num, n_top_skills=5):
    skills_in_cluster = df_skill[df_skill['cluster'] == cluster_num]['job_skills']
    all_skills = ', '.join(skills_in_cluster)
    skill_list = all_skills.split(', ')
    most_common_skills = Counter(skill_list).most_common(n_top_skills)
    return [skill for skill, freq in most_common_skills]

In [ ]:
job_title = "Software Developer"
cluster = kmeans.predict(vectorizer.transform([job_title]))[0]
top_skills = get_top_skills(cluster)

print(f"Top skills for '{job_title}': {top_skills}")

In [ ]:
technical_skills = ['JavaScript', 'Python', 'C', 'C++', 'Java', 'SQL', 'HTML', 'CSS', 'React', 'Node.js']

def extract_technical_skills(skills):
    return [skill for skill in skills.split(', ') if skill in technical_skills]

df_skill['technical_skills'] = df_skill['job_skills'].apply(extract_technical_skills)

vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(df_skill['job_title'])

num_clusters = 10
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
df_skill['cluster'] = kmeans.fit_predict(X)

def get_top_skills(cluster_num, n_top_skills=5):
    skills_in_cluster = df_skill[df_skill['cluster'] == cluster_num]['technical_skills']
    all_skills = [skill for sublist in skills_in_cluster for skill in sublist]
    most_common_skills = Counter(all_skills).most_common(n_top_skills)
    return [skill for skill, freq in most_common_skills]

In [ ]:
job_title = "Web Developer"
cluster = kmeans.predict(vectorizer.transform([job_title]))[0]
top_skills = get_top_skills(cluster)

print(f"Top skills for '{job_title}': {top_skills}")

In [ ]:
technical_skills = ['JavaScript', 'Python', 'C', 'C++', 'Java', 'SQL', 'HTML', 'CSS', 'React', 'Node.js']

vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(df_skill['job_title'])

num_clusters = 10
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
df_skill['cluster'] = kmeans.fit_predict(X)

def get_top_skills(cluster_num, n_top_skills=5):
    skills_in_cluster = df_skill[df_skill['cluster'] == cluster_num]['job_skills']
    all_skills = ', '.join(skills_in_cluster).split(', ')
    
    # Count all skills and technical skills separately
    all_skills_count = Counter(all_skills)
    tech_skills_count = Counter([skill for skill in all_skills if skill in technical_skills])

    # Combine results, favoring technical skills
    combined_count = {}
    for skill in all_skills_count:
        combined_count[skill] = all_skills_count[skill] + (tech_skills_count[skill] * 2)  # Weight technical skills more

    # Get the most common skills based on combined scoring
    most_common_skills = Counter(combined_count).most_common(n_top_skills)
    return [skill for skill, _ in most_common_skills]

In [ ]:
job_title = "Database Manager"
cluster = kmeans.predict(vectorizer.transform([job_title]))[0]
top_skills = get_top_skills(cluster)

print(f"Top skills for '{job_title}': {top_skills}")